In [ ]:
def generar_predicciones_por_cantidad(
    modelos: list,
    X_predict: pd.DataFrame,
    clientes_predict: np.ndarray,
    corte: int | list = 11000,  
    nombre_base: str = None
) -> dict:
    """
    Genera predicciones finales seleccionando un NÚMERO FIJO de clientes
    con las probabilidades más altas, guardando dos CSVs por corte:
    - Solo predicciones
    - Predicciones + probabilidades promedio
    """
    logger.info(f"Generando predicciones finales por cantidad con {len(modelos)} modelos... Corte {corte}")

    # Asegurar que cortes sea lista
    if isinstance(corte, int):
        cortes = [corte]
    else:
        cortes = corte

    try:
        # Promediar probabilidades de todos los modelos
        probabilidades_todos = [modelo.predict(X_predict) for modelo in modelos]
        probabilidades_promedio = np.mean(probabilidades_todos, axis=0)

        resultados_por_cantidad = {}

        for cantidad in cortes:
            try:
                indices_ordenados = np.argsort(probabilidades_promedio)[::-1]
                indices_top_n = indices_ordenados[:cantidad]

                # Predicción binaria según top-k
                pred_bin = np.zeros_like(probabilidades_promedio, dtype=int)
                pred_bin[indices_top_n] = 1

                # DataFrame solo predicciones
                df_pred = pd.DataFrame({
                    'numero_de_cliente': clientes_predict,
                    'Predict': pred_bin
                })

                # DataFrame predicciones + probabilidades
                df_pred_prob = df_pred.copy()
                df_pred_prob['Probabilidad'] = probabilidades_promedio

                # Guardar ambos CSVs
                nombre_archivo_pred = f"{nombre_base or STUDY_NAME}_cantidad_{cantidad}_pred"
                ruta_pred = guardar_predicciones_finales(df_pred, nombre_archivo=nombre_archivo_pred)

                nombre_archivo_pred_prob = f"{nombre_base or STUDY_NAME}_cantidad_{cantidad}_pred_prob"
                ruta_pred_prob = guardar_predicciones_finales(df_pred_prob, nombre_archivo=nombre_archivo_pred_prob)

                positivos = int(df_pred['Predict'].sum())
                porcentaje = round((positivos / len(df_pred)) * 100, 2)
                logger.info(f"Cantidad {cantidad}, {positivos:,} positivos ({porcentaje:.2f}%)")

                resultados_por_cantidad[cantidad] = {
                    'ruta_pred': ruta_pred,
                    'ruta_pred_prob': ruta_pred_prob,
                    'positivos': positivos,
                    'porcentaje': porcentaje,
                    'df_pred': df_pred,
                    'df_pred_prob': df_pred_prob
                }

            except Exception as e:
                logger.error(f"Error al procesar cantidad {cantidad}: {e}", exc_info=True)
                continue

        return {
            'probabilidades_promedio': probabilidades_promedio,
            'resultados_por_cantidad': resultados_por_cantidad
        }

    except Exception as e:
        logger.error(f"Error al generar predicciones finales: {e}", exc_info=True)
        raise
